# 🎓 Week 10 - Day 3: Regularization and Training Techniques
## Preventing Overfitting and Professional CNN Training

### Today's Learning Goals:
- ✅ Understand overfitting and how to detect it
- ✅ Implement dropout for regularization
- ✅ Use batch normalization effectively
- ✅ Apply data augmentation techniques
- ✅ Implement early stopping
- ✅ Visualize training with TensorBoard
- ✅ Compare models with/without regularization
- ✅ Build a production-ready CNN

---

**Let's master professional CNN training!** 🚀

## 📦 Part 1: Setup and Imports

Let's set up our environment with all necessary libraries!

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Set seeds
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print(f"✅ PyTorch version: {torch.__version__}")
print("✅ All libraries imported successfully!")

## 📊 Part 2: Load Data with Augmentation

Let's load Fashion-MNIST with data augmentation for training!

In [ ]:
# Training transforms WITH augmentation
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance to flip
    transforms.RandomRotation(degrees=10),     # Rotate ±10 degrees
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Slight shift
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Test transforms WITHOUT augmentation
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=train_transform
)
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=test_transform
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"✅ Training samples: {len(train_dataset):,}")
print(f"✅ Test samples: {len(test_dataset):,}")
print(f"\n🎨 Data augmentation applied to training set!")
print("   • Random horizontal flip")
print("   • Random rotation (±10°)")
print("   • Random translation")

In [ ]:
# Visualize augmented samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

# Get one image
original_img = train_dataset.dataset.data[0].numpy()

# Show augmented versions
for i in range(10):
    augmented, label = train_dataset[0]
    ax = axes[i // 5, i % 5]
    ax.imshow(augmented.squeeze().numpy(), cmap='gray')
    ax.set_title(f"Augmented {i+1}", fontsize=10)
    ax.axis('off')

plt.suptitle("Same Image with Different Augmentations", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Each epoch, images are augmented differently!")
print("   This effectively multiplies your training data!")

## 🏗️ Part 3: Models - With and Without Regularization

Let's build two models to compare regularization effects!

In [ ]:
class CNNWithoutRegularization(nn.Module):
    """Simple CNN without any regularization"""
    def __init__(self):
        super(CNNWithoutRegularization, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 28→14
        x = self.pool(F.relu(self.conv2(x)))  # 14→7
        x = self.pool(F.relu(self.conv3(x)))  # 7→3
        
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNWithRegularization(nn.Module):
    """CNN with dropout and batch normalization"""
    def __init__(self):
        super(CNNWithRegularization, self).__init__()
        
        # Conv layers with batch norm
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # FC layers with dropout
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.dropout1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        # Conv blocks with BatchNorm
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # FC layers with Dropout
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)  # Dropout before output
        x = self.fc2(x)
        return x


# Create both models
model_no_reg = CNNWithoutRegularization().to(device)
model_with_reg = CNNWithRegularization().to(device)

print("🏗️ Models Created:\n")
print("="*60)
print("Model WITHOUT Regularization:")
print(f"  Parameters: {sum(p.numel() for p in model_no_reg.parameters()):,}")
print("  • No Dropout")
print("  • No Batch Normalization")
print()
print("Model WITH Regularization:")
print(f"  Parameters: {sum(p.numel() for p in model_with_reg.parameters()):,}")
print("  • Dropout (p=0.5) in FC layers")
print("  • Batch Normalization after each Conv")
print("="*60)

## 🚂 Part 4: Training Functions with TensorBoard

Let's create training functions that log to TensorBoard!

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device, writer=None, epoch=0):
    """
    Train for one epoch with TensorBoard logging
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc='Training', leave=False)):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Log to TensorBoard every 100 batches
        if writer and batch_idx % 100 == 0:
            step = epoch * len(train_loader) + batch_idx
            writer.add_scalar('Batch/Loss', loss.item(), step)
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


def evaluate(model, test_loader, criterion, device):
    """
    Evaluate model on test set
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss = running_loss / len(test_loader)
    test_acc = 100. * correct / total
    
    return test_loss, test_acc


def train_model(model, train_loader, test_loader, num_epochs, model_name):
    """
    Complete training loop with TensorBoard logging
    """
    # Setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    writer = SummaryWriter(f'runs/{model_name}')
    
    # History
    train_losses, train_accs = [], []
    test_losses, test_accs = [], []
    
    print(f"\n🚂 Training {model_name}...\n")
    print("="*60)
    
    for epoch in range(num_epochs):
        # Train
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device, writer, epoch
        )
        
        # Evaluate
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
        # Save history
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        # Log to TensorBoard
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/test', test_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/test', test_acc, epoch)
        
        # Print progress
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Test Loss:  {test_loss:.4f} | Test Acc:  {test_acc:.2f}%")
        print("-" * 60)
    
    writer.close()
    print("\n✅ Training complete!")
    
    return train_losses, train_accs, test_losses, test_accs

print("✅ Training functions defined!")
print("✅ TensorBoard logging enabled!")
print("\n💡 Run this in terminal to view TensorBoard:")
print("   tensorboard --logdir=runs")

## 🎯 Part 5: Train Model WITHOUT Regularization

Let's train the baseline model to see overfitting!

In [ ]:
# Train model without regularization
history_no_reg = train_model(
    model_no_reg, 
    train_loader, 
    test_loader, 
    num_epochs=10,
    model_name='no_regularization'
)

train_losses_no_reg, train_accs_no_reg, test_losses_no_reg, test_accs_no_reg = history_no_reg

## 🛡️ Part 6: Train Model WITH Regularization

Now let's train the regularized model!

In [ ]:
# Train model with regularization
history_with_reg = train_model(
    model_with_reg, 
    train_loader, 
    test_loader, 
    num_epochs=10,
    model_name='with_regularization'
)

train_losses_reg, train_accs_reg, test_losses_reg, test_accs_reg = history_with_reg

## 📊 Part 7: Compare Results

Let's visualize the difference regularization makes!

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss - No Regularization
axes[0, 0].plot(train_losses_no_reg, 'b-o', label='Train Loss', linewidth=2)
axes[0, 0].plot(test_losses_no_reg, 'r-o', label='Test Loss', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_title('WITHOUT Regularization - Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Loss - With Regularization
axes[0, 1].plot(train_losses_reg, 'b-o', label='Train Loss', linewidth=2)
axes[0, 1].plot(test_losses_reg, 'r-o', label='Test Loss', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0, 1].set_title('WITH Regularization - Loss', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Accuracy - No Regularization
axes[1, 0].plot(train_accs_no_reg, 'b-o', label='Train Acc', linewidth=2)
axes[1, 0].plot(test_accs_no_reg, 'r-o', label='Test Acc', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('WITHOUT Regularization - Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Accuracy - With Regularization
axes[1, 1].plot(train_accs_reg, 'b-o', label='Train Acc', linewidth=2)
axes[1, 1].plot(test_accs_reg, 'r-o', label='Test Acc', linewidth=2)
axes[1, 1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('WITH Regularization - Accuracy', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print comparison
print("\n📊 Final Results Comparison:\n")
print("="*60)
print("WITHOUT Regularization:")
print(f"  Train Accuracy: {train_accs_no_reg[-1]:.2f}%")
print(f"  Test Accuracy:  {test_accs_no_reg[-1]:.2f}%")
print(f"  Gap: {train_accs_no_reg[-1] - test_accs_no_reg[-1]:.2f}% (overfitting!)")
print()
print("WITH Regularization:")
print(f"  Train Accuracy: {train_accs_reg[-1]:.2f}%")
print(f"  Test Accuracy:  {test_accs_reg[-1]:.2f}%")
print(f"  Gap: {train_accs_reg[-1] - test_accs_reg[-1]:.2f}% (better!)")
print("="*60)

print("\n✅ Regularization reduced overfitting!")
print("💡 Notice: Smaller gap between train and test accuracy!")

## 🎯 Challenge - Build Your Production CNN

**Your Task:** Create a production-ready CNN with:
1. Batch normalization after every conv layer
2. Dropout (0.3-0.5) in FC layers
3. Data augmentation (already done)
4. Weight decay in optimizer
5. Achieve >90% test accuracy

**Bonus:**
- Implement early stopping
- Try different architectures
- Visualize with TensorBoard


In [ ]:
# YOUR CODE HERE
# Build your production-ready CNN



## 📚 Summary & Key Takeaways

### 🎓 What We Learned:

**1. Overfitting Detection:**
- Large gap between train and test accuracy
- Train loss decreasing, test loss increasing
- Model memorizing instead of learning

**2. Regularization Techniques:**
- **Dropout:** Random neuron deactivation (0.3-0.5)
- **Batch Normalization:** Normalize activations
- **Data Augmentation:** Create variations
- **Weight Decay:** Penalize large weights (1e-4)
- **Early Stopping:** Stop before overfitting

**3. Best Practices:**
- Always use batch normalization
- Add dropout in FC layers
- Apply data augmentation
- Monitor train vs test metrics
- Save best model checkpoints

**4. TensorBoard:**
- Real-time training visualization
- Compare multiple experiments
- Track loss, accuracy, weights
- Professional ML workflow

### 💡 Pro Tips:

1. Start simple, add regularization gradually
2. BatchNorm after Conv, Dropout before output
3. Use weight_decay=1e-4 in optimizer
4. Monitor validation set closely
5. Save checkpoints every epoch
6. Visualize everything with TensorBoard

### 🎯 When to Use What:

**Small dataset (< 10K samples):**
- Heavy data augmentation
- Higher dropout (0.5)
- Smaller network

**Large dataset (> 100K samples):**
- Light augmentation
- Lower dropout (0.3)
- Deeper network OK

**Signs of overfitting:**
- Add more regularization
- Reduce model complexity
- Get more data

**Signs of underfitting:**
- Increase model capacity
- Train longer
- Reduce regularization

---

**Congratulations! You now know professional CNN training! 🎉**

**Next Steps:**
- Apply to real projects
- Experiment with architectures
- Try transfer learning
- Deploy your models!